# M3 FAULT 10

IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M3\FAULT_M3_10


SETTAGGIO DELLA VARIABILE RELATIVA AL GUASTO

In [3]:
guasto=2

## RCOU
VARIABILE RCOU POTENZA MOTORI.

Andiamo a prendere una potenza media di 1350 per considerare il momento il cui inizia/finisce il volo

In [4]:
rcou_m3_fault10 = pd.read_csv("RCOU.csv")
rcou_m3_fault10 = rcou_m3_fault10.astype("float64")
rcou_m3_fault10 = rcou_m3_fault10.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m3_fault10 = rcou_m3_fault10[((rcou_m3_fault10['C9'] >= potenza) & (rcou_m3_fault10['C10'] >= potenza) & (rcou_m3_fault10['C11'] >= 1350) & (rcou_m3_fault10['C12'] >= potenza) & (rcou_m3_fault10['C13'] >= potenza) & (rcou_m3_fault10['C14']>= potenza))]
rcou_m3_fault10=rcou_m3_fault10.reset_index(drop=True)
display(rcou_m3_fault10)

,TimeUS,C9,C10,C11,C12,C13,C14
0,106891806.0,1479.0,1406.0,1428.0,1459.0,1460.0,1427.0
1,106907540.0,1455.0,1450.0,1499.0,1401.0,1499.0,1402.0
2,106920441.0,1362.0,1542.0,1506.0,1409.0,1415.0,1501.0
3,106931177.0,1545.0,1371.0,1425.0,1502.0,1509.0,1418.0
4,106942020.0,1535.0,1398.0,1429.0,1510.0,1496.0,1444.0
...,...,...,...,...,...,...,...
6827,186281436.0,1556.0,1438.0,1522.0,1478.0,1504.0,1495.0
6828,186291822.0,1552.0,1431.0,1521.0,1467.0,1504.0,1485.0
6829,186302079.0,1521.0,1451.0,1535.0,1435.0,1493.0,1482.0
6830,186312025.0,1502.0,1450.0,1546.0,1397.0,1494.0,1458.0


Andiamo a prendere i tempi di inizio e di fine, così da poter effettuare il trimming delle altre tabelle

In [5]:
min=rcou_m3_fault10['TimeUS'][0]
max=rcou_m3_fault10['TimeUS'][len(rcou_m3_fault10)-1]
print(max)
print(min)

186322432.0
106891806.0


## ATT & XKF
### XKF
XKF utilizzato per fare poi il confronto con i valori di roll, pitch e yaw presenti att

In [6]:
xkf1_m3_fault10 = pd.read_csv("XKF1_0.csv")
xkf1_m3_fault10 = xkf1_m3_fault10.astype("float64")
xkf1_m3_fault10 = xkf1_m3_fault10.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m3_fault10 = xkf1_m3_fault10[((xkf1_m3_fault10['TimeUS'] >= min) & (xkf1_m3_fault10['TimeUS'] <= max	))]
xkf1_m3_fault10 = xkf1_m3_fault10.reset_index(drop=True)
print(xkf1_m3_fault10)

           TimeUS  Roll  Pitch    Yaw
0     106905542.0 -0.33   0.35  12.46
1     106918323.0 -0.39   0.39  12.45
2     106928766.0 -0.20   0.37  12.46
3     106939536.0 -0.15   0.37  12.46
4     106950235.0 -0.17   0.36  12.45
...           ...   ...    ...    ...
7189  186278947.0 -0.97   1.00   0.99
7190  186289650.0 -1.03   0.97   1.00
7191  186299874.0 -1.02   0.92   1.03
7192  186309774.0 -0.92   0.81   1.03
7193  186319793.0 -0.76   0.68   1.03

[7194 rows x 4 columns]


### ATT
Andiamo a prendere dal csv relativo ad ATT i dati relativi alle variabili di roll, pitch e yaw

In [7]:
att_m3_fault10 = pd.read_csv("ATT.csv")
att_m3_fault10 = att_m3_fault10.astype("float64")
att_m3_fault10 = att_m3_fault10.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m3_fault10 = att_m3_fault10[((att_m3_fault10['TimeUS'] >= min) & (att_m3_fault10['TimeUS'] <= max	))]
att_m3_fault10=att_m3_fault10.reset_index(drop=True)
print(att_m3_fault10)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     106906700.0     0.04 -0.33     -0.01   0.35   12.39  12.46
1     106919694.0     0.02 -0.39      0.00   0.39   12.39  12.45
2     106930531.0     0.00 -0.20      0.00   0.37   12.39  12.46
3     106941454.0     0.00 -0.15      0.00   0.37   12.39  12.46
4     106951490.0    -0.02 -0.17      0.00   0.36   12.39  12.45
...           ...      ...   ...       ...    ...     ...    ...
7189  186280650.0     1.31 -0.97     -0.67   1.00    4.83   0.99
7190  186291048.0     1.36 -1.03     -0.64   0.97    4.83   1.00
7191  186301226.0     1.39 -1.02     -0.59   0.92    4.83   1.03
7192  186311233.0     1.38 -0.92     -0.52   0.81    4.83   1.03
7193  186321744.0     1.33 -0.76     -0.43   0.68    4.83   1.03

[7194 rows x 7 columns]


Tramite un ciclo for si va a vedere se i valori di XKF corrispondono con quelli di ATT in caso contrario fa una media tra i due

In [8]:
from numpy import empty

if len(att_m3_fault10)>len(xkf1_m3_fault10):
    to_add=att_m3_fault10[len(xkf1_m3_fault10):]
    att_m3_fault10=att_m3_fault10[:len(xkf1_m3_fault10)]

if len(xkf1_m3_fault10)>len(att_m3_fault10):
    to_add=xkf1_m3_fault10[len(att_m3_fault10):]

for idx,i in enumerate(att_m3_fault10['Roll']):
    if(xkf1_m3_fault10['Roll'][idx] != i):
        att_m3_fault10['Roll'][idx] = ((att_m3_fault10['Roll'][idx] + xkf1_m3_fault10['Roll'][idx])/2)

for idx,i in enumerate(att_m3_fault10['Pitch']):
    if(xkf1_m3_fault10['Pitch'][idx] != i):
        att_m3_fault10['Pitch'][idx] = ((att_m3_fault10['Pitch'][idx] + xkf1_m3_fault10['Pitch'][idx])/2)

for idx,i in enumerate(att_m3_fault10['Yaw']):
    if(xkf1_m3_fault10['Yaw'][idx] != i):
        att_m3_fault10['Yaw'][idx] = ((att_m3_fault10['Yaw'][idx] + xkf1_m3_fault10['Yaw'][idx])/2)
if 'to_add' in globals():
    att_m3_fault10=pd.concat([att_m3_fault10,to_add])

print(att_m3_fault10)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     106906700.0     0.04 -0.33     -0.01   0.35   12.39  12.46
1     106919694.0     0.02 -0.39      0.00   0.39   12.39  12.45
2     106930531.0     0.00 -0.20      0.00   0.37   12.39  12.46
3     106941454.0     0.00 -0.15      0.00   0.37   12.39  12.46
4     106951490.0    -0.02 -0.17      0.00   0.36   12.39  12.45
...           ...      ...   ...       ...    ...     ...    ...
7189  186280650.0     1.31 -0.97     -0.67   1.00    4.83   0.99
7190  186291048.0     1.36 -1.03     -0.64   0.97    4.83   1.00
7191  186301226.0     1.39 -1.02     -0.59   0.92    4.83   1.03
7192  186311233.0     1.38 -0.92     -0.52   0.81    4.83   1.03
7193  186321744.0     1.33 -0.76     -0.43   0.68    4.83   1.03

[7194 rows x 7 columns]


## IMU
VALORI DI ACCELERAZIONE E DEL GIROSCOPIO

Per le 3 tabelle di IMU si sono presi i soli valori relativi a GyrX, GyrY, GyrZ, AccX, AccY, AccZ.
In seguito si è poi creata una singola tabella popolandola con la media dei valori delle tre tabelle.

In [10]:
imu_0_m3_fault10 = pd.read_csv("IMU_0.csv")
imu_0_m3_fault10 = imu_0_m3_fault10.astype("float64")
imu_0_m3_fault10 = imu_0_m3_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m3_fault10 = imu_0_m3_fault10[((imu_0_m3_fault10['TimeUS'] >= min) & (imu_0_m3_fault10['TimeUS'] <= max))]

imu_1_m3_fault10 = pd.read_csv("IMU_1.csv")
imu_1_m3_fault10 = imu_1_m3_fault10.astype("float64")
imu_1_m3_fault10 = imu_1_m3_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m3_fault10 = imu_1_m3_fault10[((imu_1_m3_fault10['TimeUS'] >= min) & (imu_1_m3_fault10['TimeUS'] <= max))]

imu_2_m3_fault10 = pd.read_csv("IMU_2.csv")
imu_2_m3_fault10 = imu_2_m3_fault10.astype("float64")
imu_2_m3_fault10 = imu_2_m3_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m3_fault10 = imu_2_m3_fault10[((imu_2_m3_fault10['TimeUS'] >= min) & (imu_2_m3_fault10['TimeUS'] <= max))]

imu_m3_fault10 = pd.concat((imu_0_m3_fault10, imu_1_m3_fault10, imu_2_m3_fault10))
imu_m3_fault10=imu_m3_fault10.groupby(imu_m3_fault10.TimeUS, as_index=False).mean()

display(imu_m3_fault10)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,106893398.0,-0.181833,-0.085428,0.024458,0.351882,0.613464,-9.740562
1,106896186.0,-0.205686,-0.056071,0.012061,0.153629,0.735929,-9.858738
2,106898723.0,-0.079848,0.040186,0.001080,0.093860,0.851389,-9.918224
3,106902129.0,0.180783,0.050828,-0.016770,0.214891,1.032160,-9.818566
4,106904697.0,0.305814,-0.087739,-0.038796,0.212236,0.727475,-9.630442
...,...,...,...,...,...,...,...
27341,186311070.0,0.089517,-0.180582,0.027792,0.618502,1.844759,-12.362567
27342,186313759.0,0.151039,-0.174859,-0.009015,0.656361,2.363123,-12.738627
27343,186316447.0,0.319917,-0.200447,-0.010393,0.736293,2.474347,-12.872283
27344,186318718.0,0.269170,-0.224830,-0.001907,0.871600,2.249914,-12.945027


## UNIONE DELLE TABELLE
Vengono unite le tabelle relative a IMU, ATT e RCOU, riempiendo i valori mancanti con quelli vicini

In [11]:
m3_fault10 = pd.merge_ordered(imu_m3_fault10,att_m3_fault10)
m3_fault10=m3_fault10.fillna(method="ffill").fillna(method="bfill")
m3_fault10 = pd.merge_ordered(m3_fault10, rcou_m3_fault10)
m3_fault10=m3_fault10.fillna(method="ffill").fillna(method="bfill")

Vengono aggiornati i tempi facendo partire il volo all'istante 0, e viene inserita la colonna relativa al guasto, valorizzandola con la variabile *guasto*.

In [12]:
m3_fault10["TimeUS"] = m3_fault10["TimeUS"] - m3_fault10.iloc[0]["TimeUS"]
m3_fault10["TimeUS"] = m3_fault10["TimeUS"].astype(int)
m3_fault10["Guasto"] = guasto

In [ ]:
last_cell = m3_fault10.iloc[-1, m3_fault10.columns.get_loc('TimeUS')]
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)
df = pd.DataFrame(to_be, columns = ['TimeUS'])

In [ ]:
m3_fault10_final = pd.merge_ordered(m3_fault10,df)
m3_fault10_final=m3_fault10_final.fillna(method="ffill").fillna(method="bfill")
m3_fault10_final = m3_fault10_final[m3_fault10_final.TimeUS.isin(to_be)]

Viene formattata la colonna del tempo in maniera tale che sia più leggibile all'operatore umano

In [19]:
from datetime import timedelta
m3_fault10_final=m3_fault10_final.reset_index(drop=True)
m3_fault10_final['TimeUS'] = pd.to_datetime(m3_fault10_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m3_fault10_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,-0.181833,-0.085428,0.024458,0.351882,0.613464,-9.740562,0.04,-0.33,-0.01,0.35,12.39,12.46,1479.0,1406.0,1428.0,1459.0,1460.0,1427.0,2.0
1,00:00:00.002857,-0.181833,-0.085428,0.024458,0.351882,0.613464,-9.740562,0.04,-0.33,-0.01,0.35,12.39,12.46,1479.0,1406.0,1428.0,1459.0,1460.0,1427.0,2.0
2,00:00:00.005714,-0.205686,-0.056071,0.012061,0.153629,0.735929,-9.858738,0.04,-0.33,-0.01,0.35,12.39,12.46,1479.0,1406.0,1428.0,1459.0,1460.0,1427.0,2.0
3,00:00:00.008571,-0.079848,0.040186,0.001080,0.093860,0.851389,-9.918224,0.04,-0.33,-0.01,0.35,12.39,12.46,1479.0,1406.0,1428.0,1459.0,1460.0,1427.0,2.0
4,00:00:00.011428,0.180783,0.050828,-0.016770,0.214891,1.032160,-9.818566,0.04,-0.33,-0.01,0.35,12.39,12.46,1479.0,1406.0,1428.0,1459.0,1460.0,1427.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27798,00:01:19.418886,0.231043,-0.164744,0.048283,0.548167,1.842138,-12.117257,1.39,-1.02,-0.59,0.92,4.83,1.03,1521.0,1451.0,1535.0,1435.0,1493.0,1482.0,2.0
27799,00:01:19.421743,0.089517,-0.180582,0.027792,0.618502,1.844759,-12.362567,1.38,-0.92,-0.52,0.81,4.83,1.03,1502.0,1450.0,1546.0,1397.0,1494.0,1458.0,2.0
27800,00:01:19.424600,0.151039,-0.174859,-0.009015,0.656361,2.363123,-12.738627,1.38,-0.92,-0.52,0.81,4.83,1.03,1502.0,1450.0,1546.0,1397.0,1494.0,1458.0,2.0
27801,00:01:19.427457,0.269170,-0.224830,-0.001907,0.871600,2.249914,-12.945027,1.38,-0.92,-0.52,0.81,4.83,1.03,1502.0,1450.0,1546.0,1397.0,1494.0,1458.0,2.0


## Analisi nel tempo e in frequenza

Funzione che calcola il Root Mean Square

In [20]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

Viene definita la funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza


In [21]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m3_fault10_final)/350)):
        V1=m3_fault10_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab = tab.astype("float64")
        tab=tab.append(val,ignore_index=True)
    return tab

Si richiama la funzione descritta sopra per ogni variabile in modo da avere le statistiche sia nel tempo che in frequenza

In [22]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
c_9 =time_freq_feat('C9')
c_10 =time_freq_feat('C10')
c_11 =time_freq_feat('C11')
c_12 =time_freq_feat('C12')
c_13 =time_freq_feat('C13')
c_14 =time_freq_feat('C14')

merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,c_9,c_10,c_11,c_12,c_13,c_14]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_30620\4244968141.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_30620\4244968141.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards 

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),var(GyrY),kurt(GyrY),RMS(GyrY),freq1(GyrY),freq2(GyrY),amp1(GyrY),amp2(GyrY),<>(GyrZ),var(GyrZ),kurt(GyrZ),RMS(GyrZ),freq1(GyrZ),freq2(GyrZ),amp1(GyrZ),amp2(GyrZ),<>(AccX),var(AccX),kurt(AccX),RMS(AccX),freq1(AccX),freq2(AccX),amp1(AccX),amp2(AccX),<>(AccY),var(AccY),kurt(AccY),RMS(AccY),freq1(AccY),freq2(AccY),amp1(AccY),amp2(AccY),<>(AccZ),var(AccZ),kurt(AccZ),RMS(AccZ),freq1(AccZ),freq2(AccZ),amp1(AccZ),amp2(AccZ),<>(Roll),var(Roll),kurt(Roll),RMS(Roll),freq1(Roll),freq2(Roll),amp1(Roll),amp2(Roll),<>(Pitch),var(Pitch),kurt(Pitch),RMS(Pitch),freq1(Pitch),freq2(Pitch),amp1(Pitch),amp2(Pitch),<>(Yaw),var(Yaw),kurt(Yaw),RMS(Yaw),freq1(Yaw),freq2(Yaw),amp1(Yaw),amp2(Yaw),<>(C9),var(C9),kurt(C9),RMS(C9),freq1(C9),freq2(C9),amp1(C9),amp2(C9),<>(C10),var(C10),kurt(C10),RMS(C10),freq1(C10),freq2(C10),amp1(C10),amp2(C10),<>(C11),var(C11),kurt(C11),RMS(C11),freq1(C11),freq2(C11),amp1(C11),amp2(C11),<>(C12),var(C12),kurt(C12),RMS(C12),freq1(C12),freq2(C12),amp1(C12),amp2(C12),<>(C13),var(C13),kurt(C13),RMS(C13),freq1(C13),freq2(C13),amp1(C13),amp2(C13),<>(C14),var(C14),kurt(C14),RMS(C14),freq1(C14),freq2(C14),amp1(C14),amp2(C14),Guasto
0,0.0,-0.011828,0.017444,0.197152,0.132418,88.0,66.0,5.799726,5.493375,-0.000067,0.004159,-0.504184,0.064398,0.0,78.0,3.832457,2.598880,0.169462,0.011011,-1.276282,0.199240,82.0,75.0,1.730703,1.191792,0.001236,0.070432,0.029097,0.265015,114.0,0.0,30.772788,7.399904,0.367656,0.090414,0.572738,0.474686,97.0,100.0,18.178429,10.619049,-10.788633,0.647120,-1.215883,10.818497,0.0,106.0,156.901914,7.314425,0.409430,0.309623,-0.858059,0.690199,88.0,80.0,3.814745,1.690608,-0.258832,0.119243,-0.953228,0.431158,0.0,105.0,69.473113,7.885594,16.396040,12.015793,-1.452952,16.757437,113.0,84.0,-2.483821,-2.918366,1587.814815,4068.037037,2.246174,1589.091669,92.0,95.0,1791.691521,1224.427921,1474.874644,4352.647098,-1.210489,1476.345306,93.0,94.0,1777.590693,1649.713218,1580.455840,2903.225902,1.195573,1581.371436,98.0,102.0,1274.785168,915.945710,1485.641026,2849.716484,-0.937599,1486.597072,106.0,108.0,1183.770681,1102.517038,1520.512821,1727.679121,-0.157561,1521.079220,100.0,97.0,1546.212831,1332.519497,1550.626781,1634.748881,3.586800,1551.152315,92.0,90.0,1619.786365,756.229777,2
1,1.0,-0.012811,0.017734,0.203216,0.133597,101.0,76.0,5.822283,4.054995,0.000341,0.004206,-0.501656,0.064761,0.0,104.0,3.905408,2.634052,0.169157,0.011012,-1.280207,0.198984,85.0,78.0,1.706437,1.002776,0.001763,0.070329,0.027888,0.264826,110.0,0.0,31.139231,7.393084,0.367834,0.090167,0.581803,0.474566,96.0,97.0,13.364875,10.657504,-10.786059,0.647608,-1.213888,10.815953,0.0,65.0,156.824762,7.736564,0.405767,0.313464,-0.833569,0.690812,101.0,97.0,3.685388,1.623960,-0.257358,0.119668,-0.969295,0.430768,0.0,105.0,70.284580,7.886682,16.412415,12.075944,-1.457326,16.775249,105.0,78.0,-2.532419,-3.175683,1588.002841,4068.891730,2.248809,1589.279823,97.0,100.0,1846.110046,1227.587636,1474.633523,4360.711466,-1.215571,1476.107161,104.0,105.0,1862.772485,1538.899324,1580.642045,2907.159253,1.195971,1581.558780,102.0,107.0,1075.964051,948.464864,1485.403409,2861.472125,-0.943099,1486.363559,103.0,101.0,1289.177900,1191.715461,1520.480114,1723.133507,-0.151720,1521.045040,100.0,97.0,1507.054426,1366.045816,1550.656250,1630.397169,3.607911,1551.180380,94.0,91.0,1521.109648,651.935860,2
2,2.0,-0.013688,0.017956,0.186585,0.134508,104.0,78.0,5.873542,4.953687,0.000722,0.004245,-0.507100,0.065066,0.0,104.0,3.968748,2.714317,0.168815,0.011022,-1.285046,0.198719,81.0,79.0,1.627865,0.711797,0.003046,0.070710,0.005789,0.265554,124.0,0.0,30.713442,7.653436,0.368467,0.090052,0.580736,0.474937,91.0,94.0,11.322662,10.904794,-10.783887,0.647435,-1.211222,10.813779,0.0,100.0,156.602139,7.603121,0.402125,0.317257,-0.811936,0.691421,92.0,69.0,3.525743,1.792204,-0.255892,0.120086,-0.985001,0.430380,0.0,111.0,71.089164,7.857893,16.428697,12.135220,-1.461714,16.792943

## Output
Salviamo la tabella ottenuta in un file csv

In [23]:
path_file = path_file.replace(r"M3\FAULT_M3_10", "")
os.chdir(path_file)
df_merged.to_csv('m3_fault10.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
